In [12]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
import os
os.listdir('./data/movie_lens')

['links.csv', 'movies.csv', 'ratings.csv', 'ratings_noh.csv', 'tags.csv']

# data 불러오기
## Dataset.load_from_file(file_path, reader)
- surprise에 os 파일을 로딩할 때는
- header = False
- userid, itemid, rating 순으로 단 3개의 칼럼만 불러옴
- 네번째 칼럼부터는 아예 로딩을 하지 않으므로 따로 지정

In [6]:
ratings = pd.read_csv('./data/movie_lens/ratings.csv')
ratings.to_csv('./data/movie_lens/ratings_noh.csv', index = False, header = False)

In [9]:
#header = False인 파일을 먼저 저장 후, 다시 읽기
#reader를 먼저 지정해주고, Dataset.load_from_file을 이용하여 부르기
reader = Reader(line_format='user item rating timestamp', #user,tiem,rating,timestamp 4개 칼럼을 명시
                sep = ',', rating_scale=(0.5,5)) #평점의 단위는 0.5, max 5)
data = Dataset.load_from_file('./data/movie_lens/ratings_noh.csv', reader = reader)

In [11]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

## Dataset.load_from_df(df, reader)

In [20]:
#header = T인 파일을 판다스로 먼저 읽고, 리더로 파일형태 지정
ratings = pd.read_csv('./data/movie_lens/ratings.csv')

reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(ratings[['userId', 'movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

# Model

## SVD

- n_factors : 잠재 요인 K의 개수.(default 100). 커질수록 정확도가 높아질 수 있으나 과적합 문제 발생
- n_epochs : SGD 수행 시 반복 횟수. (default 20)
- biased : 베이스라인 사용자 편향 적용 여부. (default True)


In [22]:
#SVD
from surprise import SVD
from surprise.accuracy import rmse

In [23]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## Baseline 평점
- 사람마다 성향에 따라 같은 영화를 봐도 주관적인 평가를 하게 됨.
- 같은 점수라도 주관적.
- 한 개인의 성향을 반영해 아이템 평가에 bias요소를 반영하여 평점을 부과하는 것.

$$ 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수$$
    - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평균을 평균한 값
    - 사용자 편향 점수 : 사용자별 아이템 평균 평점 값 - 전체 평균 평점
    - 아이템 편향 점수 : 아이템별 평점 평균 값 - 전체 평균 평점

# 교차 검증과 hyperparameter 튜닝

## cv

In [24]:
from surprise.model_selection import cross_validate

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv = 5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8739  0.8696  0.8691  0.8897  0.8669  0.8739  0.0083  
MAE (testset)     0.6732  0.6683  0.6689  0.6815  0.6652  0.6714  0.0056  
Fit time          4.94    4.79    5.08    5.26    5.25    5.06    0.18    
Test time         0.14    0.27    0.14    0.12    0.14    0.16    0.06    


{'test_rmse': array([0.87391957, 0.86961929, 0.86909272, 0.88974108, 0.86691824]),
 'test_mae': array([0.67324172, 0.6683124 , 0.66886711, 0.68146026, 0.66516453]),
 'fit_time': (4.935218811035156,
  4.794471263885498,
  5.0789947509765625,
  5.256476879119873,
  5.254803895950317),
 'test_time': (0.14059734344482422,
  0.27448296546936035,
  0.1406254768371582,
  0.12497282028198242,
  0.14059066772460938)}

## hyperparameter

In [26]:
from surprise.model_selection import GridSearchCV

#n_epochs : SGD 반복 횟수 지정 / n_factors : 잠재 요인 K의 크기
param_grid = {'n_epochs' : [20,40,60], 'n_factors' : [50,100,200]}

In [27]:
#3-fold. rmse, mae
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv = 3)
gs.fit(data)

In [28]:
print(gs.best_score['rmse'])
print(gs.best_score['mae'])

0.8771526619245907
0.6749148151860206


# Surprise를 이용한 개인화 영화 추천 시스템

- Surprise 데이터세트는 train_test_split()을 이용해 내부에서 사용하는 trainset 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수가 없다.

In [29]:
#오류 발생
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

## DatasetAutoFolds
- 전체 데이터를 학습할 수 있음

In [31]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep = ',', rating_scale=(0.5,5))

data_folds = DatasetAutoFolds(ratings_file='./data/movie_lens/ratings_noh.csv', reader = reader)

#전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [32]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [39]:
#영화 상세 속성 정보
movies = pd.read_csv('./data/movie_lens/movies.csv')
#9번 user가 본 영화 아이디 목록
moviesId = ratings[ratings['userId'] == 9]['movieId']

if moviesId[moviesId == 42].count() == 0 :
    print('사용자 9는 42 영화 평점 없음')

사용자 9는 42 영화 평점 없음


## 평점을 매기지 않은 영화의 예측 평점 구하기

In [40]:
#반드시 string 타입
uid = str(9); iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


## 예측 평점 순으로 영화 추천

In [41]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [42]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
def get_unseen_surprise(ratings, movies, userId) :
    #특정 사용자가 평점을 매긴 모든 영화
    seen_movies = ratings[ratings.userId == userId]['movieId'].tolist()
    #모든 영화 
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수 : ', len(seen_movies), '추천 대상 영화 수 : ', len(unseen_movies),
         '전체 영화 수 : ', len(total_movies))
    
    return unseen_movies

In [47]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 :  46 추천 대상 영화 수 :  9696 전체 영화 수 :  9742


- 사용자 아이디9버은 전체 9742개의 영화 중에서 46개만 평점을 매김

In [51]:
def recomm_movie_by_surprise(alog, userId, unseen_movies, top_n = 10) :
    #알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트 객체, 추천 상위 N개
    
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    #sorting을 예측한 평점 값인 est로 하기 위함
    def sortkey_est(pred) :
        return pred.est
    
    predictions.sort(key = sortkey_est, reverse = True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions ]
    top_movie_rating = [pred.est for pred in top_predictions ]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_perds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_perds

In [54]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)

print('##### TOP-10 추천 리스트 #####')
for top_movie in top_movie_preds :
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수 :  46 추천 대상 영화 수 :  9696 전체 영화 수 :  9742
##### TOP-10 추천 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
